In [ ]:
'''
breadboard_python_watchdog.py
=============================
This lets you watch a folder for new single images and upload the metadata to Breadboard. 
Usage:

python3 breadboard_python_watchdog.py [WATCHFOLDER]

where [WATCHFOLDER] is the folder your camera program writes images to.

'''

import os
import time
import datetime
import shutil
import posixpath
import sys
from breadboard import BreadboardClient
import warnings
import pandas as pd

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')


def getFileList(folder=os.getcwd()):
    # Get a list of files in a folder
    if not os.path.exists(folder):
        raise ValueError("Folder '{}' doesn't exist".format(folder))
    # Folder contents
    filenames = [filename for filename in os.listdir(folder)]
    # Output
    paths = [os.path.join(folder, f) for f in filenames]
    return (filenames, paths)

def get_newest_run_dict(run_id_guess):
    while True:
        run_dict = bc._send_message('get','/runs/' + str(run_id_guess) + '/').json()
        if 'runtime' not in run_dict.keys():
            new_run_dict = bc._send_message('get','/runs/' + str(run_id_guess - 1) + '/').json()
            new_run_dict_clean = {'runtime': new_run_dict['runtime'],
                                  'run_id': new_run_dict['id'], 
                                  **new_run_dict['parameters']}
            return new_run_dict_clean, run_id_guess - 1
        run_id_guess += 1

def main():
    # Global settings
    # bc = BreadboardClient(config_path='API_CONFIG.json') # enter your path to the API_config
    refresh_time = 0.5  # seconds
    watchfolder = os.getcwd() + '\images'  # feed the program your watchfolder
    print("\n\n Watching this folder for changes: " + watchfolder + "\n\n")

    names_old, paths_old = getFileList(watchfolder)
    df_log = None
    if os.path.exists('image_log.csv'):
        df_log = pd.read_csv('image_log.csv')
    n_images_per_run = 3
    run_id_guess = bc._send_message('get','/runs/').json()['results'][0]['id'] #209843
    # Main Loop
    
    while True:
        # Get a list of all the images in the folder
        names, paths = getFileList(watchfolder)
        names = sorted(names)

        # check if a new image has come in
        if len(names) > len(names_old):
            new_names = set(names).difference(set(names_old))
            if len(new_names) < n_images_per_run:
                #waiting for all images from  newest run
                continue
            else:
                new_names = sorted(new_names)
            
            output_filenames = new_names[0:3]

            # associate new image with latest run parameters in local log

            new_row_dict, run_id_guess = get_newest_run_dict(run_id_guess)
            for filenum in range(n_images_per_run):
                new_row_dict['filename' + str(filenum)] = output_filenames[filenum]
            if df_log is None:
                for key in new_row_dict.keys():
                    new_row_dict[key] = [new_row_dict[key]]
                df_log = pd.DataFrame(new_row_dict)
            else:
                if os.path.exists('image_log.csv'):
                    df_log = pd.read_csv('image_log.csv')
                df_log = df_log.append(new_row_dict, ignore_index=True)
            df_log.to_csv('image_log.csv')

            # # Write to Breadboard
            # try:
            #     resp = bc.post_images(
            #                 image_names = os.path.splitext(output_filename)[0],
            #                 filepath = output_filepath,
            #                 image_times = [dt],
            #                 auto_time = False # Add more information here
            #                 )
            #     if resp.status_code!=200:
            #         print(resp.text)
            # except: pass
            # names_old = names
            names_old = names
            print('New file(s): ' + str(output_filenames) + ' associated with run parameters \n')
            print(new_row_dict['runtime'])
            listboundvars = new_row_dict['ListBoundVariables']
            try:
                for var in listboundvars:
                    print(var + ':' + str(new_row_dict[var]))
            except:
                print('bug: first shot formatting issues')
                
        # Wait before checking again
        time.sleep(refresh_time)


main()




 Watching this folder for changes: C:\Users\FermiCam2\Documents\FermiData2\2020\Jul\20200722\images


New file: test_103.spe associated with run parameters 

2020-07-22T16:00:41Z
dummy1:-0.1
New file: test_104.spe associated with run parameters 

2020-07-22T16:01:00Z
dummy1:-0.1
New file: test_105.spe associated with run parameters 

2020-07-22T16:01:19Z
dummy1:-0.1
New file: test_106.spe associated with run parameters 

2020-07-22T16:01:38Z
dummy1:-0.1
New file: test_107.spe associated with run parameters 

2020-07-22T16:01:57Z
dummy1:-0.1
New file: test_108.spe associated with run parameters 

2020-07-22T16:02:15Z
dummy1:-0.1
New file: test_109.spe associated with run parameters 

2020-07-22T16:02:34Z
dummy1:-0.1
New file: test_110.spe associated with run parameters 

2020-07-22T16:02:53Z
dummy1:-0.1
New file: test_111.spe associated with run parameters 

2020-07-22T16:03:12Z
dummy1:-0.1
New file: test_112.spe associated with run parameters 

2020-07-22T16:03:31Z
dummy1:-0.1
New fi